# Wordle helper 

Simple Julia code to help with playing the game Wordle. 

## Loading the code

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

import DataStructures: counter
import Random: shuffle 

include(joinpath(@__DIR__, "wordle_helper.jl"))

  Activating project at `~/Github/Wordle_helper`


solve (generic function with 1 method)

## Finding the best starting word

In [2]:
let 
    all_words = get_word_lists()
    find_best_guess(;all_words..., verbose = true)
end 

Progress: 100%|█████████████████████████████████████████| Time: 0:02:39


("roate", 139883)

Using only guesses that could be solutions:

In [3]:
let 
    all_words = get_word_lists(; use_all_allowed_guesses = false)
    find_best_guess(;all_words..., verbose = true)
end 

Progress: 100%|█████████████████████████████████████████| Time: 0:00:27


("raise", 141216)

## Solving some words

Testing some solutions

In [4]:
let 
    all_words = get_word_lists(; use_all_allowed_guesses = false)
    for w in ["epoxy", "lodge", "wound", "awash", "raise"] 
        println(solve(w, "roate"; all_words...))
    end 
end

["roate", "demon", "epoch", "epoxy"]
["roate", "slump", "lodge"]
["roate", "scold", "blimp", "awash", "wound"]
["roate", "slick", "admin", "awash"]
["roate", "raise"]


### Hard mode

In [5]:
let 
    all_words = get_word_lists(; use_all_allowed_guesses = false)
    for w in ["epoxy", "lodge", "wound", "awash", "raise"] 
        println(solve(w, "roate"; all_words..., hard_mode = true))
    end
end  

["roate", "demon", "epoch", "epoxy"]
["roate", "louse", "lodge"]
["roate", "godly", "bound", "found", "hound", "mound", "pound", "sound", "wound"]
["roate", "slack", "awash"]
["roate", "raise"]


## Solving all games

In [6]:
function solve_all_words(; use_all_allowed_guesses = true, hard_mode = false, first_guess = "roate")
    all_words = get_word_lists(; use_all_allowed_guesses)
    p = Progress(length(all_words.words))
    return ThreadsX.map(shuffle(all_words.words)) do word 
        next!(p)
        solve(word, first_guess; all_words..., hard_mode)
    end
end

solve_all_words (generic function with 1 method)

In [7]:
let 
    sol = solve_all_words()
    counter(map(x -> length(x), sol)) |> collect |> sort
end 

Progress: 100%|█████████████████████████████████████████| Time: 0:08:36


4-element Vector{Pair{Int64, Int64}}:
 2 => 53
 3 => 1112
 4 => 1100
 5 => 50

This algorithm breaks all Wordle puzzles under 5 guesses. 

Using only guesses that are potential solutions: 

In [8]:
let 
    sol = solve_all_words(; use_all_allowed_guesses = false, first_guess = "raise")
    counter(map(x -> length(x), sol)) |> collect |> sort
end

Progress: 100%|█████████████████████████████████████████| Time: 0:01:30


5-element Vector{Pair{Int64, Int64}}:
 1 => 1
 2 => 60
 3 => 1066
 4 => 1095
 5 => 93

### Hard mode

In [9]:
let 
    sol = solve_all_words(; hard_mode = true)
    counter(map(x -> length(x), sol)) |> collect |> sort
end 

Progress: 100%|█████████████████████████████████████████| Time: 0:00:04


8-element Vector{Pair{Int64, Int64}}:
 2 => 126
 3 => 980
 4 => 954
 5 => 199
 6 => 43
 7 => 9
 8 => 3
 9 => 1

Using only guesses that are potential solutions:

In [10]:
let 
    sol = solve_all_words(; hard_mode = true, use_all_allowed_guesses = false, first_guess = "raise")
    counter(map(x -> length(x), sol)) |> collect |> sort
end 

Progress: 100%|█████████████████████████████████████████| Time: 0:00:03


8-element Vector{Pair{Int64, Int64}}:
 1 => 1
 2 => 131
 3 => 944
 4 => 949
 5 => 233
 6 => 43
 7 => 11
 8 => 3

## Breaking one Wordle

### Wordle 220

In [11]:
all_words = get_word_lists()
game_words = get_word_lists().words

new_guess_and_update!(game_words, "raise", [1, 1, 0, 1, 0], all_words.words)

("chant", 49)

In [12]:
new_guess_and_update!(game_words, "chant", [0, 0, 1, 0, 0], all_words.words)

("solar", 2)

In [13]:
new_guess_and_update!(game_words, "solar", [2, 0, 0, 2, 2], all_words.words)

("sugar", 0)

### Wordle 223 

In [14]:
all_words = get_word_lists()
game_words = get_word_lists().words

new_guess_and_update!(game_words, "raise", [1, 0, 0, 0, 1], all_words.words)

("outer", 885)

In [15]:
new_guess_and_update!(game_words, "outer", [0, 0, 0, 1, 1], all_words.words)

("clerk", 25)

In [16]:
new_guess_and_update!(game_words, "clerk", [0, 0, 1, 1, 1], all_words.words)

("jerky", 1)

In [17]:
new_guess_and_update!(game_words, "jerky", [0, 2, 2, 2, 2], all_words.words)

("perky", 0)

### Hard mode. 


Wordle 222 

In [18]:
all_words = get_word_lists()
game_words = get_word_lists().words

new_guess_and_update!(game_words, "raise", [0, 0, 0, 0, 0], game_words, hard_mode = true)

("could", 1053)

In [19]:
new_guess_and_update!(game_words, "could", [0, 2, 2, 0, 0], game_words, hard_mode = true)

("mouth", 6)

In [20]:
new_guess_and_update!(game_words, "mouth", [2, 2, 2, 1, 0], game_words, hard_mode = true)

("mount", 0)